In [ ]:
!apt-get install unrar
!pip install rarfile

In [ ]:
import rarfile

rarfile.UNRAR_TOOL = "/usr/bin/unrar"
!wget https://github.com/repositoriosHackaton/SIC25es-Remember-Us-Recuerdanos-/raw/refs/heads/main/recursos/dataset.rar

rar_path = "/content/dataset.rar"  # Ruta del archivo RAR
extract_path = "dataset"  # Carpeta de salida

with rarfile.RarFile(rar_path) as rar_ref:
    rar_ref.extractall(extract_path)

print("Archivo descomprimido correctamente.")

In [ ]:
#algunas importaciones
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
ruta_origen = "dataset/dataset/Training"
ruta_smoking = "dataset/dataset/training/smoking"
ruta_notsmoking = "dataset/dataset/training/notsmoking"

os.makedirs(ruta_smoking, exist_ok=True)
os.makedirs(ruta_notsmoking, exist_ok=True)

for archivo in os.listdir(ruta_origen):
  if os.path.isfile(os.path.join(ruta_origen, archivo)):
    if "notsmoking" not in archivo.lower() :
            shutil.move(os.path.join(ruta_origen, archivo), os.path.join(ruta_smoking, archivo))
    elif(True):
            shutil.move(os.path.join(ruta_origen, archivo), os.path.join(ruta_notsmoking, archivo))


datadir = "/content/dataset/dataset/training"
imgsize = (224, 224)
batchsize = (32)

datagen = ImageDataGenerator(rescale= 1/255, validation_split=0.2)

train_data = datagen.flow_from_directory(datadir,target_size=imgsize,
                                         batch_size=batchsize,class_mode='binary',subset='training')
val_data = datagen.flow_from_directory(datadir,target_size=imgsize,
                                       batch_size=batchsize,class_mode='binary',subset='validation')

In [ ]:
from tensorflow.keras import layers, regularizers

# Modelo base Xception
mnet = keras.applications.Xception(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
mnet.trainable = True
for layer in mnet.layers[:-5]:
    layer.trainable = False

model = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),

    mnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-3)),
    layers.BatchNormalization(),
    layers.Dropout(0.8),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=15,
        restore_best_weights=True
    )
]

# Entrenamiento
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=60,
    callbacks=callbacks
)
# Guardar el modelo entrenado
model.save("modelo_xception.h5")
